### Aplicação do Algoritmo e Criação da Matriz

 - Aqui pra não vamos comparar todos os registros um a um, vamos usar duas colunas **COMPLEMENTO** e **LOGRADOURO**.
 - Vamos tirar uma amostra aleatória (6mil) da base A e vamos comparar par a par as colunas **COMPLEMENTO** e **LOGRADOURO** de ambas as bases. Caso contrario, teríamos 4*( (100mil) * (6mil) ) comparações a se fazer.
 - Esse método irá diminuir o custo do processo.

Imports

In [136]:
import numpy as np
import pandas as pd
import jellyfish 

Extração das Bases

In [138]:
#Extraindo as duas bases "Log_Comp_df" faz parte da base B
df_A = pd.read_csv("./Bases_Geradas/baseA.csv", na_values = "None", keep_default_na= False)
Log_Comp_df = pd.read_csv("./Bases_Geradas/Log_Comp.csv", na_values = "None", keep_default_na= False)

In [140]:
#Selecionamos somente as colunas "Nome_Logradouro" e "Complemento"
df_A = df_A.iloc[:][["Nome_Logradouro","Complemento"]].sample(replace = False, n = 6000, random_state = 42)

In [141]:
Log_Comp_df.head(10)

,Nome_Logradouro,Complemento
0,FONTE DO BOI,APARTAMENTO 303 BLOCO B
1,OSWALDO CRUZ,
2,FONTE DO BOI,APARTAMENTO ANDAR
3,BARRO VERMELHO,APARTAMENTO 204 BLOCO B
4,BARRO VERMELHO,APARTAMENTO 105
5,AUGUSTO SEVERO,
6,TAMOIOS,ANDAR 201
7,ARGOLO,
8,MAMUTE,
9,VINHAIS,APARTAMENTO 201


In [142]:
df_A.head(10)

,Nome_Logradouro,Complemento
75721,PAULO RUBENS,ANDAR 1
80184,CLARA,TERREO
19864,EULER DE PEREIRA CARDOSO,CASA
76699,CLAUDIONILDA,ANDAR 1
92991,MARTACENIA,ANDAR 2
76434,DO SOSSEGO DA SARAMANDAIA,ANDAR 1
84004,DO AMPARO,
80917,RIO BRANCO,TERREO
60767,DO LIMAO,
50074,ARTENIO COSTA VALENTE,CASA


Aplicando o Algoritmo

- Aqui usamos 2 Loops pra iterar sobre a Matriz

In [143]:
#Criação da matriz
matriz_similaridade = np.empty((6000, 2))

#Loop iterar sobre as colunas
for i in range (2): 
    for j in range (6000):  #Loop iterar sobre as linhas
        #Aqui aplicamos o algoritmo e atribuimos na matriz, arrendodamos o valor pra 3 casas decimais
        matriz_similaridade[j][i] = round((jellyfish.jaro_similarity(df_A.iloc[j][i], Log_Comp_df.iloc[j][i])),3)


In [144]:
#Transformando matriz em um Dataframe Pandas
df_similaridade = pd.DataFrame(matriz_similaridade, columns=['Coluna1A_Coluna1B', 'Coluna2A_Coluna2B'])

In [145]:
df_similaridade.head()

,Coluna1A_Coluna1B,Coluna2A_Coluna2B
0,0.472,0.577
1,0.356,0.000
2,0.528,0.539
3,0.488,0.577
4,0.505,0.613


Exportando

In [146]:
df_similaridade.to_csv("./Bases_Geradas/Matriz_Similaridade.csv")

Descritiva


- O algoritmo de **Jaro-Winkler** quando comparado duas string retorna o nível de similaridade desse par, se for totalmente diferente atribui-se **0**, se totalmente iguais **1**, os valores variam nessa faixa.

- No nosso exemplo, usamos ele por par de coluna: **Nome_Logradouro** da **base A** com **Nome_Logradouro** da **base B** e **Complemento** da **base A** com **Complemento** da **base B**

- Então na nossa matriz, que virou df, temos os valores correspondentes par a par dessas comparações, podemos ver que tem vários valorações com 0, isso porque existem diversas strings vazias no conjunto, o que normalmente leva a esse valor.

In [180]:
(df_similaridade["Coluna1A_Coluna1B"] == 1).sum()

3

In [181]:
(df_similaridade["Coluna2A_Coluna2B"] == 1).sum()

95

- Aqui vimos que há 3 valores iguais na primeira coluna (Logradouro) e 95 na segunda (Complemento)

In [182]:
(df_similaridade["Coluna1A_Coluna1B"] == 0).sum()

856

In [183]:
(df_similaridade["Coluna2A_Coluna2B"] == 0).sum()

3966

- E pra strings totalmente diferentes temos, 856 na primeira coluna (Logradouro) e 3966 na segunda coluna (Complemento)